In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15755922
paper_name = 'gardocki_lopes_2005' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [11]:
original_data = pd.read_excel('raw_data/120_GENES_AFFECTING_PIS1.xlsx', sheet_name='mat_alpha_061101', skiprows=5)

In [12]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4830 x 16


In [13]:
original_data.head()

,record no.,ORF name,strain,batch,row/col,re-array,re-array.1,re-array.2,RG notes,QC Notes,PIS1-LACZ,INO1-LACZ,TCM1-LACZ,PIS1-LACZ.1,INO1-LACZ.1,TCM1-LACZ.1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10338.0,YAL068C,BY4739,01_1,A2,101.0,A,2,NaN,NaN,+,NT,+,-,NT,NT
2,10339.0,YAL067C,BY4739,01_1,A3,101.0,A,3,NaN,NaN,-,NT,NT,-,NT,NT
3,10340.0,YAL066W,BY4739,01_1,A4,101.0,A,4,NaN,NaN,-,NT,NT,-,NT,NT
4,10341.0,YAL065C,BY4739,01_1,A5,101.0,A,5,NaN,NaN,-,NT,NT,-,NT,NT


In [14]:
original_data['orf'] = original_data['ORF name'].astype(str)

In [15]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [18]:
original_data.loc[original_data['orf']=='YYKL138C','orf'] = 'YKL138C'

In [19]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [20]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

             record no. ORF name  strain batch row/col  re-array re-array.1  \
index_input                                                                   
0                   NaN      NaN     NaN   NaN     NaN       NaN        NaN   
3628            14652.0      NaN  BY4742  07_3      H8     139.0          E   
4668                NaN      NaN     NaN   NaN     NaN       NaN        NaN   
4829                NaN      NaN     NaN   NaN     NaN       NaN        NaN   

            re-array.2 RG notes QC Notes PIS1-LACZ INO1-LACZ TCM1-LACZ  \
index_input                                                              
0                  NaN      NaN      NaN       NaN       NaN       NaN   
3628                 9      NaN      NaN         -        NT        NT   
4668           end 150      NaN      NaN         -        NT        NT   
4829           end 171      NaN      NaN       NaN       NaN       NaN   

            PIS1-LACZ.1 INO1-LACZ.1 TCM1-LACZ.1  orf  
index_input              

In [21]:
original_data = original_data.loc[t,:]

In [22]:
original_data.set_index('orf', inplace=True)

In [24]:
original_data = original_data[['PIS1-LACZ','PIS1-LACZ.1']].copy()

In [25]:
# Transform symbols to numbers
# Glucose: + -> -1, - -> 0
# Glycerol: - -> 0, + -> +1

In [27]:
original_data['data1'] = original_data['PIS1-LACZ'].apply(lambda x: -1 if x == '+' else 0)
original_data['data2'] = original_data['PIS1-LACZ.1'].apply(lambda x: 1 if x == '+' else 0)

In [28]:
original_data = original_data[['data1','data2']].copy()

In [29]:
original_data = original_data.groupby(original_data.index).mean()

In [30]:
original_data.shape

(4750, 2)

# Prepare the final dataset

In [31]:
data = original_data.copy()

In [32]:
dataset_ids = [575, 576]
datasets = datasets.reindex(index=dataset_ids)

In [33]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [34]:
data.head()

dataset_id,575,576
data_type,value,value
orf,,
YAL002W,0.0,0.0
YAL004W,0.0,0.0
YAL005C,0.0,0.0
YAL007C,0.0,0.0
YAL008W,0.0,0.0


## Subset to the genes currently in SGD

In [35]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 27


In [36]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,575,576
,data_type,value,value
gene_id,orf,,
2,YAL002W,0.0,0.0
1863,YAL004W,0.0,0.0
4,YAL005C,0.0,0.0
5,YAL007C,0.0,0.0
6,YAL008W,0.0,0.0


# Normalize

In [37]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [38]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [39]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,575,576,575,576
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,0.0,0.0,0.0,0.0
1863,YAL004W,0.0,0.0,0.0,0.0
4,YAL005C,0.0,0.0,0.0,0.0
5,YAL007C,0.0,0.0,0.0,0.0
6,YAL008W,0.0,0.0,0.0,0.0


# Print out

In [40]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [41]:
from IO.save_data_to_db3 import *

In [42]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 15755922...
Inserting the new data...


100%|██████████| 2/2 [00:15<00:00,  7.65s/it]

Updating the data_modified_on field...
